## Run de novo motif analysis on different subsets of TSS.
## 1. Extract the TSS of the regions of interest from the bed file using the Tissues in the meta file. 
### Subsets:
## Unique:
### 1. TSS unique to a tissue. 
### 2. TSS in a tissue with more than 2SD more than the median. (Mean absolute deviance)
## Houekeeping
### 3. TSS in housekeeping genes (max/median < 2) 
    

In [1]:
## Parameters specific to where your folders are and your data
parameter_file = '../../parameters/params.yaml'
import yaml
import sys
import os
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

#data_folder = doc['data_folder']
#tissues = doc['tissues'].split(',')
#sys.path.append(doc['pipeline_path'])
ref_fa = doc['GENOME_FA']
#anno_gff=doc['annotation']
#mRNA_peak_file = doc["mRNA_peak_file"]
#tss_annotation = doc['tss_annotation']

from scipy.stats import zscore

import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from os.path import join
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
from collections import defaultdict
from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
#from Homer import *

homer_path =  "/data2/resources/software/homer/v4.11_10242019/bin/"
os.environ['PATH'] = f"{homer_path}:{os.environ['PATH']}"
!which homer

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


/data2/resources/software/homer/v4.11_10242019/bin//homer


In [2]:
import click

In [3]:
from tss.utils import wrap_findMotifs
from tss.data.data_io import read_bed_file
from tss.data.data_io import write_bed_file
from tss.utils import wrap_findMotifs
from tss.data import data_io
from tss.visualize.fig_utils import helper_save
from tss.data.tissues_collapse import merge_columns


params {'PREFIX': 'ncbi_pice_103', 'ROOT_DIR': '/data/isshamie/TSS/', 'DATA_DIR': '/data/isshamie/TSS/NCBI_PICR_103_Processed/', 'DATA_PROCESSED_DIR': '/data/isshamie/TSS/TSS/data/processed/', 'SUPPLEMENTAL_DIR': '/data/isshamie/TSS/Analysis/supplemental/', 'PIPELINE': '/data/isshamie/TSS/tss', 'GENOME_DIR': '/data/isshamie/genome', 'GENOME_FA': '/data/isshamie/genome/picr.fa', 'GENOME_GFF3': '/data/isshamie/genome/updated_final_sort.gff3', 'GENOME_GTF': '/data/isshamie/genome/updated_final_sort.gtf', 'TISSUES': 'BMDM1hKLA,BMDMwt,Brain,FemaleReproductive,Heart,Intestine,', 'META_FILE': '/data/isshamie/TSS/TSS/parameters/raw_metaSamples.tsv'}


In [4]:
os.chdir(doc["results"])
genome="alt"
outdir= join("de_novo_motifs",genome,'enhancers_unique')

if not os.path.exists(outdir):
    os.makedirs(outdir)

## Outdir:

In [5]:
unique_gene_peaks_dir = "unique_genes/"
unique_dir = "/data/isshamie/TSS/Analysis/TSS_code/figures/alt/barplots_eTSS/bmdm/" #gene_tissues_expressedLiver.uniqueGenes.txt"")

## Define unique tissues

In [6]:
os.chdir(doc["results"])
genome="alt"
outdir= join("de_novo_motifs",genome)
data_dir = join(doc["results"], genome)
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
enh_dir = join(outdir,'enhancer_unique')
if not os.path.exists(enh_dir):
    os.mkdir(enh_dir)

In [7]:
os.chdir(doc["results"])
TSS1_bed = read_bed_file('alt/eTSS/TSS.exp.bed')
TSS1_meta = pd.read_csv('alt/eTSS/TSS.exp.meta.tsv',sep="\t",index_col=0)
unique_genes_dir = "enhancer_unique_genes"
silent_bed_f = 'alt/eTSS/TSS1.ref.bed'


In [8]:
tissue_expr_f = join(data_dir,"tissues_collapse/tissues.merge.peaksexpression")
peak_anno_f = join(data_dir, "merged","peaks_with_tss_distances_size1.peak")
merged_f = join(data_dir, "merged","samples.merge")

## Load peak and expression

In [9]:
tissue_expr = pd.read_csv(tissue_expr_f,sep='\t', index_col=0)
#tissue_expr = np.log2(tissue_expr+1)
tissue_expr.head()

,MiscOrgans,Liver,Brain,Lung,Heart,CHO,Pancreas,BMDM1hKLA,FemaleReproductive,Muscle,Kidney,Spleen,BMDMwt
0,0.705,0.0,0.790000,2.180000,0.835,26.996,0.000,84.2,0.630,0.715,2.130000,0.000,0.00
1,3.975,1.1,1.608571,16.596667,0.000,0.000,0.000,0.0,4.265,0.545,1.050000,3.385,0.00
2,0.705,0.0,1.887429,2.386667,0.000,19.044,1.490,0.0,4.025,1.100,1.870000,1.255,5.66
3,3.025,0.0,3.914286,1.633333,0.835,1.680,2.385,0.0,6.000,1.495,1.153333,2.875,2.45
4,112.050,29.7,189.308571,50.966667,25.600,20.292,95.500,0.0,136.550,124.150,43.640000,88.550,3.39


In [10]:
peak_anno = pd.read_csv(peak_anno_f,sep='\t', index_col=0)
peak_anno

,Chr,Start,End,Strand,Stat,Annotation,Nearest TSS,Distance to TSS,isSameStrand,Nearest gene
ID,,,,,,,,,,
0,NW_020822452.1,16590385,16590385,+,16.227333,promoter-TSS (ID=id181373;Parent=rna15526;Dbxr...,XM_027399668.1,-462.0,False,Clic4
1,NW_020822597.1,23595396,23595396,+,5.740000,Intergenic,XM_027420351.1,-23850.0,True,Baz2b
2,NW_020822680.1,728718,728718,+,7.579579,promoter-TSS (ID=id609942;Parent=rna53731;Dbxr...,XM_027432478.1,746.0,False,Irak1
3,NW_020822602.1,10883205,10883205,+,3.992000,Intergenic,XM_027423054.1,9079.0,True,Gbgt1
4,NW_020822601.1,37427907,37427907,+,129.031738,promoter-TSS (ID=id474636;Parent=rna41512;Dbxr...,XM_027421990.1,-11.0,True,Shf
...,...,...,...,...,...,...,...,...,...,...
540122,NW_020822538.1,6114468,6114468,+,7.122000,promoter-TSS (ID=id398770;Parent=rna34727;Dbxr...,XM_027415967.1,-58.0,False,Selplg
540123,NW_020822487.1,8747242,8747242,+,7.508000,Intergenic,XM_027405626.1,-7856.0,True,Herpud1
540124,NW_020822407.1,2824603,2824603,+,3.612167,Intergenic,XM_027390727.1,-132495.0,True,Klf3


In [11]:
merged_df = pd.read_csv(merged_f,sep='\t', index_col=0)
merged_df.head()

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Chr,Start,End,Strand,Stat,Parent files,Total subpeaks,MiscOrgans_START1_and_MiscOrgans_startinput1,Liver_START2_and_Liver_startinput1,Brain_GROCap1_and_Brain_GRO1,...,MiscOrgans_START2_and_MiscOrgans_startinput1,Spleen_START1_and_Spleen_startinput1,Lung_START2_and_Lung_startinput1,CHO_START1_and_CHO_startinput1,FemaleReproductive_START1_and_FemaleReproductive_startinput1,Kidney_START2_and_Kidney_startinput1,Kidney_GROCap1_and_Kidney_GRO1,CHO_GROCap1_and_CHO_GRO1,BMDMwt_GROCap1_and_BMDMwt_GRO1,Spleen_START2_and_Spleen_startinput1
ID,,,,,,,,,,,,,,,,,,,,,
0,NW_020822452.1,16590187,16590584,+,16.227333,MiscOrgans_START1_and_MiscOrgans_startinput1|B...,15,NW_020822452.1-1136,NaN,NW_020822452.1-1139,...,NaN,NaN,NaN,NW_020822452.1-450,NaN,NaN,NW_020822452.1-419,NW_020822452.1-145,NaN,NaN
1,NW_020822597.1,23595304,23595489,+,5.740000,MiscOrgans_START1_and_MiscOrgans_startinput1|B...,16,NW_020822597.1-387,NaN,NW_020822597.1-324,...,NW_020822597.1-103,NW_020822597.1-248,NW_020822597.1-10,NaN,NW_020822597.1-101,NaN,NaN,NaN,NaN,NW_020822597.1-126
2,NW_020822680.1,728607,728829,+,7.579579,MiscOrgans_START1_and_MiscOrgans_startinput1|B...,19,NW_020822680.1-92,NaN,NW_020822680.1-114,...,NaN,NaN,NaN,NW_020822680.1-37,NW_020822680.1-56,NaN,NW_020822680.1-67,NW_020822680.1-19,NW_020822680.1-44,NW_020822680.1-72
3,NW_020822602.1,10883074,10883337,+,3.992000,MiscOrgans_START1_and_MiscOrgans_startinput1|B...,20,NW_020822602.1-204,NaN,NaN,...,NW_020822602.1-254,NW_020822602.1-122,NaN,NW_020822602.1-226,NaN,NaN,NaN,NW_020822602.1-303,NW_020822602.1-594,NW_020822602.1-266
4,NW_020822601.1,37427816,37427999,+,129.031738,MiscOrgans_START1_and_MiscOrgans_startinput1|B...,23,NW_020822601.1-18,NaN,NW_020822601.1-1445,...,NW_020822601.1-52,NW_020822601.1-123,NaN,NW_020822601.1-41,NaN,NaN,NW_020822601.1-1949,NW_020822601.1-992,NW_020822601.1-2287,NW_020822601.1-12


## Average the BMDM samples

In [12]:
BMDM = tissue_expr.columns.str.contains("BMDM")

In [13]:
bmdm_map = defaultdict(list)
bmdm_map['BMDM'] = ['BMDMwt','BMDM1hKLA']
for i in tissue_expr.columns:
    if i not in bmdm_map['BMDM']:
        bmdm_map[i] = [i]
tissue_expr = merge_columns(tissue_expr,bmdm_map)
tissue_expr

,BMDM,MiscOrgans,Liver,Brain,Lung,Heart,CHO,Pancreas,FemaleReproductive,Muscle,Kidney,Spleen
0,42.100,0.705,0.0,0.790000,2.180000,0.835,26.996,0.000,0.630,0.715,2.130000,0.000
1,0.000,3.975,1.1,1.608571,16.596667,0.000,0.000,0.000,4.265,0.545,1.050000,3.385
2,2.830,0.705,0.0,1.887429,2.386667,0.000,19.044,1.490,4.025,1.100,1.870000,1.255
3,1.225,3.025,0.0,3.914286,1.633333,0.835,1.680,2.385,6.000,1.495,1.153333,2.875
4,1.695,112.050,29.7,189.308571,50.966667,25.600,20.292,95.500,136.550,124.150,43.640000,88.550
...,...,...,...,...,...,...,...,...,...,...,...,...
540122,1.520,0.890,0.0,0.000000,0.920000,0.000,0.000,1.565,0.000,0.000,0.000000,12.450
540123,11.500,0.645,0.0,0.000000,1.123333,0.000,0.000,0.940,0.000,0.000,0.000000,4.000
540124,2.440,0.000,0.0,0.000000,2.093333,0.000,0.000,0.000,0.000,0.000,0.314333,4.785
540125,1.320,3.475,0.0,0.461714,0.893333,0.000,0.260,0.000,0.000,0.000,0.000000,0.000


## Create distal peaks and get zscore

In [14]:
print(peak_anno.shape)
print(tissue_expr.shape)

(540127, 10)
(540127, 12)


In [15]:
#(peak_anno["Distance to TSS"] < -2000) &
distal_index =  (peak_anno["Annotation"].fillna('').str.contains("Intergenic")) & (tissue_expr.max(axis=1) >= 1) & (np.abs((peak_anno["Distance to TSS"] > 2000)))
#distal_index =  (peak_anno["Annotation"].fillna('').str.contains("Intergenic")) & (np.abs((peak_anno["Distance to TSS"] > 2000)))

print(distal_index.sum())
distal_index

184940


ID
0         False
1         False
2         False
3          True
4         False
          ...  
540122    False
540123    False
540124    False
540125    False
540126     True
Length: 540127, dtype: bool

In [16]:
distal_anno = peak_anno[distal_index]
distal_expr = tissue_expr[distal_index]
distal_merged = merged_df[distal_index]
distal_anno


,Chr,Start,End,Strand,Stat,Annotation,Nearest TSS,Distance to TSS,isSameStrand,Nearest gene
ID,,,,,,,,,,
3,NW_020822602.1,10883205,10883205,+,3.992000,Intergenic,XM_027423054.1,9079.0,True,Gbgt1
10,NW_020822567.1,46712264,46712264,-,3.665000,Intergenic,XM_027418086.1,29793.0,True,Sctr
15,NW_020822505.1,5449423,5449423,-,2.476000,Intergenic,XM_027408794.1,86859.0,False,Tpcn2
20,NW_020822438.1,19361255,19361255,-,37.450500,Intergenic,XM_027395681.1,137002.0,True,Larp7
27,NW_020822529.1,28068900,28068900,+,3.591714,Intergenic,XM_027412515.1,59919.0,True,Nfkbiz
...,...,...,...,...,...,...,...,...,...,...
540087,NW_020822526.1,7767101,7767101,+,11.712222,Intergenic,XM_027411731.1,18004.0,True,Snx19
540089,NW_020822529.1,50722923,50722923,+,2.578333,Intergenic,XM_027412895.1,34275.0,True,Tmem44
540111,NW_020822420.1,8626257,8626257,+,6.926063,Intergenic,XM_027393079.1,14018.0,False,Bcan


In [38]:
all_unique_distal = distal_expr[(distal_expr==0).sum(axis=1) == distal_expr.shape[1]-1]
for tis in all_unique_distal.columns.values:
    tis_out = join(enh_dir, tis + '.uniqueGenes')
    tis_peak_out = tis_out+'.tsv'
    
    print(tis_out)
    curr_distal_index = all_unique_distal[all_unique_distal[tis] > 0].index
    curr_anno = peak_anno.loc[curr_distal_index]
    print(curr_anno.shape)
    
    curr_anno.to_csv(tis_peak_out,sep='\t')
    wrap_findMotifs.run_findMotifs(tis_peak_out, tis_out, ref_fa, args=None,  bg=None)

    #(all_unique_distal>0).sum()

de_novo_motifs/alt/enhancer_unique/BMDM.uniqueGenes
(44128, 10)
nohup findMotifsGenome.pl de_novo_motifs/alt/enhancer_unique/BMDM.uniqueGenes.tsv /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna de_novo_motifs/alt/enhancer_unique/BMDM.uniqueGenes -size 200 -len 6,8,10 -p 4  > de_novo_motifs/alt/enhancer_unique/BMDM.uniqueGenes.log
de_novo_motifs/alt/enhancer_unique/MiscOrgans.uniqueGenes
(2353, 10)
nohup findMotifsGenome.pl de_novo_motifs/alt/enhancer_unique/MiscOrgans.uniqueGenes.tsv /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna de_novo_motifs/alt/enhancer_unique/MiscOrgans.uniqueGenes -size 200 -len 6,8,10 -p 4  > de_novo_motifs/alt/enhancer_unique/MiscOrgans.uniqueGenes.log
de_novo_motifs/alt/enhancer_unique/Liver.uniqueGenes
(10498, 10)
nohup findMotifsGenome.pl de_novo_motifs/alt/enhancer_unique/Liver.uniqueGenes.tsv /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna de_novo_motifs/alt/enhancer_uniqu